In [1]:
###el bueno
import scipy.io
import numpy

import torchtext
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchtext.vocab import Vectors, GloVe
import torch.nn.functional as F
import pdb
from copy import deepcopy

In [2]:
##DATA
#predicates: is_a, has_a
#subjects: (animal,bird, fish, canary, eagle, shark, salmon)
#objects: (breathes, can fly, can swim, can sing, has claws, can bite, is pink)
data = torch.Tensor(2,7,7).zero_()
##is_a
data[0,:,:7] = torch.eye(7)
data[0,:,0] = torch.ones(7,1)
data[0,3,1], data[0,4,1], data[0,5,2], data[0,6,2] = 1,1,1,1 
##has_a
data[1,:,0] = torch.ones(1,7)
data[1,1,1], data[1,2,2] = 1,1
data[1,3:5,1] = torch.ones(2,1) 
data[1,5:,2] = torch.ones(2,1)
data[1,3:,3:] = torch.eye(4)

##For Sparse
#data[0,6,0],data[1,6,0],data[1,6,2],data[1,6,6] = 0,0,0,0  #It knows [0,6,2] ... salmon is fish
#data[0,6,0],data[1,6,0],data[0,6,2],data[1,6,6] = 0,0,0,0  #It knows [1,6,2] ... salmon can swim

#data[0,6,0],data[0,6,2],data[1,6,6] = 0,0,0  #It knows [1,6,2],[1,6,0] ... salmon can swim and breathe

##Auxiliar, very small
#data = torch.Tensor(2,2,2)
#data[0,:,:] = torch.eye(2,2)
#data[0,1,0] = 1
#data[1,:,0]= torch.Tensor([1,1])




num_predicates = data.size()[0]
num_subjects = data.size()[1]
num_objects = data.size()[2]





In [3]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(num_predicates * num_subjects * num_objects, hidden_size_decoder)
        self.decode = nn.Linear(hidden_size_decoder, num_predicates * num_subjects * num_objects)
    
    def forward(self,t):
        t = t.view(-1)
        z = self.linear1(t)
        z = F.relu(z)
        z = Variable(torch.ones(1,hidden_size_decoder))  ###The encoder is not doing anything
        t0 = self.decode(z)
        t0 = F.sigmoid(t0)
        t0 = t0.view(num_predicates, num_subjects, num_objects)
        return t0

In [15]:
def decoder_efficient(valuation):

    ##Unifications
    rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
    rules_aux = rules_aux.repeat(num_predicates,1)
    embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)
    #embeddings_aux = F.dropout(embeddings_aux, p=.1,training=True)
    #rules_aux = F.dropout(rules_aux, p=.1, training=True)

    #unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
    
    unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
    unifs = torch.exp(-unifs)
    unifs_sum = torch.sum(unifs, 0)
    unifs= unifs/unifs_sum
    
    #unifs=Variable(torch.Tensor([[1,0,1,1,1,0],[0,1,0,0,0,1]]))


    valuation_new = Variable(torch.Tensor(valuation.size()))
    for predicate in intensional_predicates:
        for s in range(num_subjects):
            for o in range(num_objects):
                valuation_aux = Variable(torch.Tensor([0]))
                for body1 in range(num_predicates):
                    for body2 in range(num_predicates):
                        num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                        num = torch.max(num)

                        ## max across three rules
                        new = Variable(torch.Tensor([0]))
                        for rule in range(num_rules): 
                            unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                            new = torch.max(new,unif)
                            #could be amalgamate

                        num = num*new 
                        
                        valuation_aux = amalgamate(valuation_aux, num)
                        
                        #if predicate == 0 and s==1 and o==1 and valuation_aux.data[0]>.3:
                        #    print('body1', body1, 'body2', body2)

                valuation_new[predicate,s,o] = amalgamate(valuation[predicate,s,o], valuation_aux)

    return valuation_new
    
def amalgamate(x,y):
    return x + y - x*y

In [16]:

num_iters = 100
learning_rate = .001
steps = 2
hidden_size_decoder = 500


num_feat = 2
num_rules = 2
intensional_predicates = [0,1]
num_intensional_predicates = len(intensional_predicates)

#embeddings = Variable(torch.eye(num_predicates), requires_grad=True)
embeddings = Variable(torch.rand(num_predicates, num_feat), requires_grad=True)

rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#rule1 = torch.Tensor([1,0,1,0,1,0]).view(1,-1)
#rule2 = torch.Tensor([0,1,1,0,0,1]).view(1,-1)
#rules = Variable(torch.cat((rule1,rule2),0), requires_grad=True)

encoder_m = Encoder()


optimizer = torch.optim.Adam([
        {'params': encoder_m.parameters()},
        {'params': [rules]}
    ], lr = learning_rate)

criterion = torch.nn.BCELoss(size_average=False)


#beta = 1./data.nelement()
beta = 1./data.nelement()
lamb= 1.5/data.nelement()
x = Variable(data)
print('x', x)

for epoch in range(num_iters):
    optimizer.zero_grad()
    
    valuation = encoder_m(x)
    
    #loss_reg = torch.sum(torch.ge(valuation,0.4).type(torch.FloatTensor))
    loss_reg = torch.sum(valuation)
    
    #if epoch % 49 == 0 and epoch>0 :
    print('epoch {} before_decoder'.format(epoch), valuation)
        
    for step in range(steps):
        valuation = decoder_efficient(valuation)
        if epoch % 39 == 0 and epoch>0 :
            print('epoch {}, step {}'.format(epoch,step+1), valuation)
    
    #loss = lamb2*( criterion(valuation[1,6,0],x[1,6,0]) + criterion(valuation[0,6,2], x[0,6,2]) + criterion(valuation[0,6,6], x[0,6,6]) )
    #loss += beta*criterion(valuation[:,:-1,:-1],x[:,:-1,:-1]) + lamb*loss_reg
    loss = beta*criterion(valuation,x) + lamb*loss_reg 
    
    
    print(epoch, 'losssssssssssssssssssss',loss.data[0])
    loss.backward()
    optimizer.step()

#data[0,6,0],data[1,6,0],data[1,6,2],data[1,6,6] = 0,0,0,0
# Knows: salmon is fish, salmon is salmon




('x', Variable containing:
(0 ,.,.) = 
   1   0   0   0   0   0   0
   1   1   0   0   0   0   0
   1   0   1   0   0   0   0
   1   1   0   1   0   0   0
   1   1   0   0   1   0   0
   1   0   1   0   0   1   0
   1   0   1   0   0   0   1

(1 ,.,.) = 
   1   0   0   0   0   0   0
   1   1   0   0   0   0   0
   1   0   1   0   0   0   0
   1   1   0   1   0   0   0
   1   1   0   0   1   0   0
   1   0   1   0   0   1   0
   1   0   1   0   0   0   1
[torch.FloatTensor of size 2x7x7]
)
('epoch 0 before_decoder', Variable containing:
(0 ,.,.) = 
  0.3915  0.3507  0.4867  0.4581  0.3799  0.4070  0.6297
  0.2678  0.6573  0.5517  0.2519  0.6138  0.4772  0.3941
  0.6014  0.6179  0.7536  0.7782  0.4720  0.2961  0.3895
  0.6181  0.5029  0.5943  0.6361  0.5433  0.3057  0.5789
  0.7357  0.7247  0.2877  0.6134  0.3973  0.5859  0.6725
  0.6727  0.4534  0.7667  0.7403  0.3661  0.6674  0.6365
  0.5072  0.3648  0.3676  0.5248  0.4396  0.6483  0.6064

(1 ,.,.) = 
  0.2510  0.3990  0.7070  0.5231  

In [17]:
print(embeddings)
print(rules)

rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
rules_aux = rules_aux.repeat(num_predicates,1)
embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)


unifs_real = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
print('aaaaa',unifs_real)
#print('oooo',F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1))

unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
unifs = torch.exp(-unifs)
#print(unifs)
unifs_sum = torch.sum(unifs, 0)
unifs= unifs/unifs_sum
print('finaaaal',unifs)


Variable containing:
 0.4575  0.1115
 0.2619  0.2587
[torch.FloatTensor of size 2x2]

Variable containing:
 0.4431  0.0175  0.9016  0.1334  0.8083  0.7960
 0.7539  0.3217  0.3889  0.5281  0.7217  0.4674
[torch.FloatTensor of size 2x6]

('aaaaa', Variable containing:
 0.9801  0.9865  0.9957  0.7668  0.8584  0.9442
 0.7387  0.9302  0.8066  0.9877  1.0000  0.9792
[torch.FloatTensor of size 2x6]
)
('finaaaal', Variable containing:
 0.5514  0.5331  0.5516  0.4690  0.4993  0.5154
 0.4486  0.4669  0.4484  0.5310  0.5007  0.4846
[torch.FloatTensor of size 2x6]
)


In [20]:
rules[0][0]

Variable containing:
 1
[torch.FloatTensor of size 1]